In [1]:
from sklearn.datasets import load_breast_cancer

In [2]:
data = load_breast_cancer()

In [3]:
len(data.feature_names)

30

In [4]:
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='|S23')

In [5]:
len(data.data[:,2])


569

In [6]:
len(data.data[1,:])

30

In [7]:
min(data.data[:,2])


43.79

In [8]:
sum(data.target)

357

In [9]:
#print(data.data[:5])
#print(data.data[:,3])

In [10]:
import numpy as np

In [11]:
median=np.median(data.data[:,3])
print(median)

551.1


In [12]:
import math
#熵的函数
def H(x):
    x=np.array(x)
    n=len(x)
    m=sum(x)
    if(m==0 or m==n):
        return 0
    ans=0
    p=m/n
    ans-=p*math.log(p,2)
    p=1-p
    ans-=p*math.log(p,2)
    return ans

根据各属性的中值划分成二叉树，并由此根据信息增益来判断合适的划分所在的列

In [13]:
def condition_entropy(dataset,axis):
    median=np.median(dataset[:,axis])
    left=[]
    right=[]
    for x in dataset:
        if(x[axis]<median):
            left.append(x)
        else :
            right.append(x)
    if(len(left)==0 or len(right)==0):
        return 0
    n=len(dataset)
    left=np.array(left)
    
    #print(np.shape(left))
    h1=H(left[:,-1])*len(left)/n
    right=np.array(right)
    h2=H(right[:,-1])*len(right)/n
    return h1+h2

In [14]:
def min_condition_entropy_index(dataset,axis_n):
    index=-1
    min_condition_entropy=0
    dataset=np.array(dataset)
    for axis in range(axis_n):
        value=condition_entropy(dataset,axis)
        if(index==-1):
            index=axis
            min_condition_entropy=value
        if(value<min_condition_entropy):
            index=axis
            min_condition_entropy=value
    return index

In [15]:
#根据给定的列数据的中值将数据集划分成两个子数据集
def divide(dataset,axis):
    dataset=np.array(dataset)
    median=np.median(dataset[:,axis])
    left=[]
    right=[]
    for x in dataset:
        if(x[axis]<median):
            left.append(x)
        else :
            right.append(x)
    return left,right

数据的预处理

In [16]:
def combine(x,y):
    dataset=[]
    n=len(y)
    
    for i in range(n):
        t=[]
        t=np.append(t,x[i])
        t=np.append(t,y[i])
        dataset.append(t)
    return dataset
        
#x=data.data[:5]
#y=data.target[:5]
#print(data.data[:5])
dataset=combine(data.data,data.target)
dataset=np.array(dataset)
#print(dataset[:5])

划分数据集，将数据集划分成训练集和测试集

In [17]:
import random
def split(x,y):
    n=len(x)
    t=[i for i in range(n)]
    random.shuffle(t)
    x=x[t]
    y=y[t]
    m=int(0.7*n)
    train_x=x[:m]
    train_y=y[:m]
    test_x=x[m:]
    test_y=y[m:]
    return train_x,train_y,test_x,test_y

In [18]:
train_x,train_y,test_x,test_y=split(data.data,data.target)

In [19]:
print(len(train_x))

398


In [20]:
train_dataset=combine(train_x,train_y)


In [21]:
index=min_condition_entropy_index(np.array(dataset),30)

In [22]:
left,right=divide(dataset,index)
np.shape(left)

(284, 31)

In [23]:
len(left)+len(right)

569

In [24]:
class Tree:
    def __init__(self,axis,median,target=-1):
        self.axis=axis
        self.median=median
        self.target=target
        self.left=None
        self.right=None
        

In [25]:
def create_tree(dataset,axis_n=30,min_node=10):
    n=len(dataset)
    dataset=np.array(dataset)
    m=sum(dataset[:,-1])
    
    if(n<min_node):
        if(m>n/2):
            node=Tree(0,0,1);
        else:
            node=Tree(0,0,0);
        return node
    
    
    if(m==0 or n==m):
        node = Tree(0,0,dataset[0,-1])
        return node
    
    axis=min_condition_entropy_index(dataset,axis_n)
    median=np.median(dataset[:,axis])
    node = Tree(axis,median)
    #print(axis,median,len(dataset))
    left,right=divide(dataset,axis)
    if(len(left)>0):
        node.left=create_tree(left,axis_n)
    if(len(right)>0):
        node.right=create_tree(right,axis_n)
    return node

root=create_tree(dataset)


In [26]:
#给定数据，给出类别
def search(x,axis_n=30):
    node=root
    while (node.target==-1):
        axis=node.axis
        median=node.median
        if(x[axis]<median):
            node=node.left
        else:
            node=node.right
    return node.target

#dataset[:,-1]


In [27]:
def predict(data):
    y=[]
    for x in data:
        t=search(x)
        y.append(t)
    return y
y_=predict(dataset)

In [28]:
def acc(y,y_):
    
    y=np.array(y)
    y_=np.array(y_)
    n=len(y)
    same=0
    for i in range(n):
        #print(y[i],y_[i])
        if y[i]==y_[i]:
            
            same+=1
    return same
dataset=np.array(dataset)
acc(dataset[:,-1],y_)

549

In [29]:
def cross_validation(dataset,min_node=10,k_folds=5):
    n=len(dataset)
    m=int((n+k_folds-1)/k_folds)
    #print(n)
    total_acc=0
    for k in range(k_folds):
        val_left=k*m
        val_right=min((k+1)*m,n)
        s=u'现在验证集的范围为'
        #print(s)
        #print('[%d,%d'%val_left val_right,')')
        #print(val_left,val_right)
        train_data=[]
        val_data=[]
        for i in range(val_left,val_right):
            val_data.append(dataset[i])
        for i in range(val_left):
            train_data.append(dataset[i])
            
        for i in range(val_right,n):
            train_data.append(dataset[i])
        #print(len(train_data))
        root=create_tree(train_data,min_node=min_node)
        y_=predict(val_data)
        train_data=np.array(train_data)
        val_data=np.array(val_data)
        t=acc(y_,val_data[:,-1])
        total_acc+=t
        #print(acc(y_,val_data[:,-1]))
    return total_acc
cross_validation(train_dataset)

380

In [30]:
def grid_search(dataset,min_node):
    best_ans=0;
    best_ans_index=0
    for node in min_node:
        t=cross_validation(dataset,min_node=node)
        print(node,t)
        if(t>best_ans):
            best_ans=t
            best_ans_index=node
            
    return best_ans_index
min_node=[3,5,7,10,15]
best_index=grid_search(train_dataset,min_node)
root=create_tree(train_dataset,min_node=best_index)


(3, 380)
(5, 380)
(7, 380)
(10, 380)
(15, 380)


In [31]:
test_y_=predict(test_x)

In [32]:
print(len(test_y_))

171


In [33]:
acc(test_y,test_y_)*1.0/len(test_y_)*100

95.32163742690058

使用sklearn的方法，跟自己写的所得的结果做比较


In [34]:
from sklearn.model_selection import KFold

In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
from sklearn.tree import DecisionTreeClassifier

In [37]:
from sklearn.metrics import make_scorer

In [38]:
def fit_5_fold(x,y):
    k_fold=KFold(n_splits=5)
    clf=DecisionTreeClassifier()
    params = {'max_depth':range(1,21),'criterion':np.array(['entropy'])}
    scoring_fnc = make_scorer(acc)
    grid = GridSearchCV(clf, param_grid=params,scoring=scoring_fnc,cv=k_fold)
    grid = grid.fit(x, y)
    return grid.best_estimator_

In [39]:
clf=fit_5_fold(train_x,train_y)

/Users/liuzexuan/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [40]:
y_=clf.predict(test_x)

In [41]:
acc(test_y,y_)*1.0/len(test_y_)*100

92.98245614035088

结论：通过上面的多次运行的结果比较，虽然由于数据集采用了random.shuffle，使得结果并不是每一次都相同，但是大体上，自己写的方法和sklearn的方法两者得到的结果相差不大